In [1]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import glob
import torch.nn as nn
import os.path as osp
import torchvision
import matplotlib.pyplot as plt
import torch
#import rotateImage
import cv2
import numpy as np
#import split_folders
import random
import torch.backends.cudnn as cudnn
import torch.optim as optim
from random import choice
from collections import OrderedDict

In [3]:
#/content/Car-damage/carPartModel-305.pth
#/content/drive/My Drive/Car-part-damage/carPartModel-305.pth
model1 = torch.load('/content/drive/My Drive/Car-part-damage/carPartModel-305.pth',map_location=torch.device('cpu'))
#model1 = torch.load('/content/Car-damage/carPartModel-305.pth')
for names in model1:
  print(names)
#for name,param in model1.items():
#   if name=='state_dict':
#     print(param)

epoch
state_dict
class_to_idx
confusion_matrix


In [7]:
cm = torch.tensor([[ 90.,   0.,   0.,   2.,   2.,   0.,   0.,   1.,   3.,   0.,   0.],
        [  1., 113.,   0.,   2.,   1.,   0.,   1.,   1.,   1.,   1.,   5.],
        [  0.,   2., 110.,   3.,   2.,   1.,   3.,   3.,   0.,   0.,   4.],
        [  4.,   0.,   2., 102.,   2.,   0.,   2.,   0.,   0.,   1.,   0.],
        [  2.,   0.,   0.,   3., 102.,   2.,   1.,   0.,   1.,   2.,   0.],
        [  2.,   1.,   0.,   1.,   0.,  68.,   1.,   2.,   0.,   7.,   0.],
        [  0.,   0.,   6.,   2.,   2.,   0., 115.,   3.,   1.,   0.,   2.],
        [  1.,   0.,   3.,   0.,   1.,   3.,   8., 136.,   0.,   2.,   8.],
        [  1.,   2.,   0.,   1.,   1.,   0.,   1.,   0., 112.,   1.,   1.],
        [  0.,   2.,   1.,   0.,   1.,   7.,   1.,   1.,   0.,  85.,   4.],
        [  0.,   1.,   6.,   3.,   0.,   0.,   1.,   5.,   2.,   1.,  84.]])
cm = cm.numpy()
print(model1["class_to_idx"])

def precision(label, confusion_matrix):
    col = confusion_matrix[:, label]
    return confusion_matrix[label, label] / col.sum()

def recall(label, confusion_matrix):
    row = confusion_matrix[label, :]
    return confusion_matrix[label, label] / row.sum()
    
def precision_macro_average(confusion_matrix):
    rows, columns = confusion_matrix.shape
    sum_of_precisions = 0
    for label in range(rows):
        sum_of_precisions += precision(label, confusion_matrix)
    return sum_of_precisions / rows

def recall_macro_average(confusion_matrix):
    rows, columns = confusion_matrix.shape
    sum_of_recalls = 0
    for label in range(columns):
        sum_of_recalls += recall(label, confusion_matrix)
    return sum_of_recalls / columns
print(" ")
print("label precision recall")
for label in range(10):
    print(f"{label:5d} {precision(label, cm):9.3f} {recall(label, cm):6.3f}")
print(" ")
print("precision total:", precision_macro_average(cm))
print("recall total:", recall_macro_average(cm))
print("")
def accuracy(confusion_matrix):
    diagonal_sum = confusion_matrix.trace()
    sum_of_all_elements = confusion_matrix.sum()
    return diagonal_sum / sum_of_all_elements

print("accuracy:",accuracy(cm))

{'glassShatterFront': 0, 'unknown': 1, 'tailLamp': 2, 'glassShatterRear': 3, 'glassShatterSide': 4, 'doorScratch': 5, 'headLamp': 6, 'bumperScratch': 7, 'multipleDamage': 8, 'doorDent': 9, 'bumperDent': 10}
 
label precision recall
    0     0.891  0.918
    1     0.934  0.897
    2     0.859  0.859
    3     0.857  0.903
    4     0.895  0.903
    5     0.840  0.829
    6     0.858  0.878
    7     0.895  0.840
    8     0.933  0.933
    9     0.850  0.833
 
precision total: 0.8717991980639371
recall total: 0.8735195560888811

accuracy: 0.8740219


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#from PIL import Image, ImageFile
#img = Image.open("/content/drive/My Drive/IMG-20200729-WA0036.jpg")


In [ ]:
#!unzip "/content/drive/My Drive/carDamageDetection.zip" -d '/content/drive/My Drive/carDamageDetection'

In [4]:
def load_model(checkpoint_path):
  chpt = torch.load(checkpoint_path)
  net = torchvision.models.resnet50(pretrained=True)
  fc = nn.Sequential(OrderedDict([
    ("dropout", nn.Dropout(0.4)),
    ("out", nn.Linear(2048, 11))]))
  net.fc = fc

  for param, x in net.named_parameters():
    if param.startswith("fc.out"):
        x.requires_grad = True
    else:
        x.requires_grad = False

  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  #net.class_to_idx = classes
  net = net.to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
    
  net.class_to_idx = chpt['class_to_idx']       
  net.load_state_dict(chpt['state_dict'])  
  return net

In [5]:
the_model = load_model('/content/drive/My Drive/Car-part-damage/carPartModel-305.pth')
# /content/drive/My Drive/carDamageDetection.zip

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


In [ ]:
dummy_input = torch.randn(1, 3, 224, 224, device='cuda')
onnx_path =  "/content/pytorch_ONNx.onnx"

torch.onnx.export(the_model, dummy_input, onnx_path, verbose=False)
print('Model exported to ' + onnx_path)

Model exported to /content/pytorch_ONNx.onnx


In [8]:
dummy_input = torch.randn(1, 3, 224, 224, device='cuda')

In [9]:
#PyTorch Profiler
import torch.autograd.profiler as profiler

In [10]:
with profiler.profile(record_shapes=True) as prof:
    with profiler.record_function("model_inference"):
        the_model(dummy_input)

In [11]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=20))

---------------------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  
Name                         Self CPU total %  Self CPU total   CPU total %      CPU total        CPU time avg     Number of Calls  
---------------------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  
model_inference              48.36%           60.957ms         100.00%          126.044ms        126.044ms        1                
conv2d                       3.26%            4.109ms          22.15%           27.916ms         526.710us        53               
convolution                  0.19%            245.032us        18.89%           23.806ms         449.178us        53               
_convolution                 0.83%            1.049ms          18.69%           23.561ms         444.555us        53               
cudnn_convolution            11.38%           14.338ms         17.76%      

In [12]:
with profiler.profile(profile_memory=True, record_shapes=True) as prof:
    the_model(dummy_input)

print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=20))

---------------------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  
Name                         Self CPU total %  Self CPU total   CPU total %      CPU total        CPU time avg     CPU Mem          Self CPU Mem     CUDA Mem         Self CUDA Mem    Number of Calls  
---------------------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  
conv2d                       1.34%            208.726us        43.73%           6.830ms          128.877us        0 b              0 b              43.33 Mb         0 b              53               
convolution                  1.45%            226.232us        42.39%           6.622ms          124.938us        0 b              0 b              43.33 Mb         0 b              53               

In [13]:
print(prof.key_averages().table(sort_by="cpu_memory_usage", row_limit=20))

---------------------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  
Name                         Self CPU total %  Self CPU total   CPU total %      CPU total        CPU time avg     CPU Mem          Self CPU Mem     CUDA Mem         Self CUDA Mem    Number of Calls  
---------------------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  
conv2d                       1.34%            208.726us        43.73%           6.830ms          128.877us        0 b              0 b              43.33 Mb         0 b              53               
convolution                  1.45%            226.232us        42.39%           6.622ms          124.938us        0 b              0 b              43.33 Mb         0 b              53               

In [ ]:
#!unzip '/content/Car-damage/splitTrainValnew.zip' -d '/content/Car-damage/splitTrainValnew' 

In [2]:
import tensorflow.compat.v1 as tf
from tensorflow.python.platform import gfile

C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is 

In [4]:
def load_model():
    with tf.gfile.GFile(path, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())


    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name="")
    return graph

if __name__=='__main__':

  #  path = '/content/drive/My Drive/Car-part-damage/carPartDamageTensorflow (1).pb'
    path = r"\Users\M1051057\Downloads\carPartDamageTensorflow (1).pb"
    graph = load_model()
    with tf.Session(graph=graph) as sess:
        for op in graph.get_operations():
            print(op.name)

bn1.bias
bn1.running_mean
bn1.running_var
bn1.weight
conv1.weight
fc.out.bias
fc.out.weight
layer1.0.bn1.bias
layer1.0.bn1.running_mean
layer1.0.bn1.running_var
layer1.0.bn1.weight
layer1.0.bn2.bias
layer1.0.bn2.running_mean
layer1.0.bn2.running_var
layer1.0.bn2.weight
layer1.0.bn3.bias
layer1.0.bn3.running_mean
layer1.0.bn3.running_var
layer1.0.bn3.weight
layer1.0.conv1.weight
layer1.0.conv2.weight
layer1.0.conv3.weight
layer1.0.downsample.0.weight
layer1.0.downsample.1.bias
layer1.0.downsample.1.running_mean
layer1.0.downsample.1.running_var
layer1.0.downsample.1.weight
layer1.1.bn1.bias
layer1.1.bn1.running_mean
layer1.1.bn1.running_var
layer1.1.bn1.weight
layer1.1.bn2.bias
layer1.1.bn2.running_mean
layer1.1.bn2.running_var
layer1.1.bn2.weight
layer1.1.bn3.bias
layer1.1.bn3.running_mean
layer1.1.bn3.running_var
layer1.1.bn3.weight
layer1.1.conv1.weight
layer1.1.conv2.weight
layer1.1.conv3.weight
layer1.2.bn1.bias
layer1.2.bn1.running_mean
layer1.2.bn1.running_var
layer1.2.bn1.weight